<a href="https://colab.research.google.com/github/spazewalker/R-C3D_tensorflow/blob/master/R_C3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# R-C3D Paper Implementation

Here We'll be implementing the R-C3D Paper which can be found [here](https://arxiv.org/pdf/1703.07814.pdf). According to the paper we'll specify 3 subparts of the model. But first let's include all the dependencies.

In [0]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [0]:
import tensorflow as tf

import os
import datetime
import tensorflow_datasets as tfds

In [0]:
# Clear any logs from previous runs
!rm -rf ./logs/ 

* And configure the notebook to use TPU

In [4]:
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
    tf.config.experimental_connect_to_cluster(resolver)
    # This is the TPU initialization code that has to be at the beginning.
    tf.tpu.experimental.initialize_tpu_system(resolver)
    strategy = tf.distribute.experimental.TPUStrategy(resolver)
except:
    print("TPU Not Available. Try changing runtime")

INFO:tensorflow:Initializing the TPU system: grpc://10.57.39.226:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.57.39.226:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [6]:
# !wget http://ai2-website.s3.amazonaws.com/data/Charades_v1_rgb.tar
!wget --continue http://ec2-52-25-205-214.us-west-2.compute.amazonaws.com/files/activity_net.v1-3.min.json
! pip install youtube-dl

--2020-05-17 14:49:42--  http://ec2-52-25-205-214.us-west-2.compute.amazonaws.com/files/activity_net.v1-3.min.json
Resolving ec2-52-25-205-214.us-west-2.compute.amazonaws.com (ec2-52-25-205-214.us-west-2.compute.amazonaws.com)... 52.25.205.214
Connecting to ec2-52-25-205-214.us-west-2.compute.amazonaws.com (ec2-52-25-205-214.us-west-2.compute.amazonaws.com)|52.25.205.214|:80... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [7]:

import json
import os
import sys

annotation_file = open('activity_net.v1-3.min.json')
annotation = json.load(annotation_file)

video_database = annotation['database']
videos = annotation['database'].keys()

# videos?

# Download the ActivityNet videos into the ./videos folder
command1 = 'mkdir '+'videos'
os.system(command1)

count = 0
for i in videos:
    count=count+1
    url = video_database[i]['url']
    command3 = 'youtube-dl -o '+'videos/'+i+' '+url
    os.system(command3)
    sys.stdout.write("\r%d out of %d downloaded"% (count,len(videos)))
    

     |████████████████████████████████| 1.8MB 3.4MB/s 


dict_keys(['sJFgo9H6zNo', 'V1zhqaGFY2A', 'JDg--pjY5gg', 'KsFid_YVsn0', '-TmWR_keSfI', 'u2uoYvo8J5s', '0Gr4aKQzGYk', 'a-6rpItrRSk', '--0edUL8zmA', 'c_NlYvL96y0', 'hHiPEAiYKv0', 'GCtrfXIBbwA', 'rs7er4e67ec', 'PLek2e8NlKc', '_uOfIm1tFcI', '0n3VRoYYYGU', 'x90zpDHTpjQ', 'T_q3f10pkOg', '6ZabZdqdLyA', 'UMVYt6Bgrrs', 'yUSo_xjyGhM', 'Ou24uqaFRPg', 'Uw_1iNGntZU', 'sjyZWmvTGA4', 'c5dvRUBZw2Q', 'BxzOUJbq28s', 'Z8yNbpjUSLc', 'A1EflBqBv14', '6uhLrPgbpUA', 'jIQFVSymHQs', '2U0vMYnC49c', 't2wawfMaTzM', 'gKkk82LG8H0', 'R-hsYQNtnn0', 'UvuXGKesWS0', 'thpmgolDsbQ', '8pR6NBu7Ay0', 'mh_0QLZhrSY', 'jhbqzV3tbDk', 'NTKUu3gpIr8', 'pRwx9my8-is', 'S2qF3yvM_PA', '-YjGbsbDoxs', 'p800u2wCKbE', 'BfnM0eyjB5Q', 'RTnNxbG2V5o', '-Xl95IW5H_s', '8bSCVzFXjzk', 'maXU1lGguxs', 'R4ES1QLRvtg', 'oD8sGLt2FR0', '9pk3tgrILeY', 'boWMu522Pfc', 'UkA6pgt29VI', 'AwcM25G1h-g', 'jafEh82XETQ', '0YQPGAsZPgY', 'akrKZpGXeo4', '5Bo0gFXxDQk', 'hmT9H0IjiGs', '97McCuWAynA', 'WEohpavjsIY', '3joaQzU05MY', 'kcV3HTdtgxk', '3X9dtfeV1mQ', 'br-L1PS2ADg',

In [47]:
import os
import json

fps = 25

# VIDEO_DIR = './videos/'
# video_list = os.listdir(VIDEO_DIR)

META_FILE = './activity_net.v1-3.min.json'
meta_data = json.load(open(META_FILE))
meta_data['database']



# mkdir('./frames')
# def generate_frame(split):
#   mkdir('./frames/%s' % split)
#   for vid, vinfo in meta_data['database'].iteritems():
#     if vinfo['subset'] == split:
#       vname = [s for s in video_list if vid in s]
#       if len(vname) != 0 :
#         filename = VIDEO_DIR + vname[0]
#         duration = vinfo['duration']
#         outpath = './frames/%s/%s/' % (split, vid)
#         outfile = outpath + "image_%5d.jpg"
#         rm(outpath)
#         mkdir(outpath)
#         ffmpeg(filename, outfile, fps)
#         for framename in os.listdir(outpath):
#           resize(outpath + framename)
#         frame_size = len(os.listdir(outpath))
#         print(filename, duration, fps, frame_size)

# generate_frame('training')
# generate_frame('validation')

{'sJFgo9H6zNo': {'annotations': [{'label': 'Fun sliding down',
    'segment': [0.01, 123.42336739937599]}],
  'duration': 139.042,
  'resolution': '640x360',
  'subset': 'training',
  'url': 'https://www.youtube.com/watch?v=sJFgo9H6zNo'},
 'V1zhqaGFY2A': {'annotations': [],
  'duration': 136.98,
  'resolution': '658x480',
  'subset': 'testing',
  'url': 'https://www.youtube.com/watch?v=V1zhqaGFY2A'},
 'JDg--pjY5gg': {'annotations': [{'label': 'Beer pong',
    'segment': [61.420442338881465, 64.56014561469672]}],
  'duration': 126.178,
  'resolution': '1920x1080',
  'subset': 'validation',
  'url': 'https://www.youtube.com/watch?v=JDg--pjY5gg'},
 'KsFid_YVsn0': {'annotations': [],
  'duration': 235.13,
  'resolution': '1920x1080',
  'subset': 'testing',
  'url': 'https://www.youtube.com/watch?v=KsFid_YVsn0'},
 '-TmWR_keSfI': {'annotations': [],
  'duration': 104.955,
  'resolution': '1280x720',
  'subset': 'testing',
  'url': 'https://www.youtube.com/watch?v=-TmWR_keSfI'},
 'u2uoYvo8J5s

In [39]:
!ls

activity_net.v1-3.min.json  Charades_v1_rgb.tar.1  README.txt.1  videos
Charades_v1_rgb.tar	    README.txt		   sample_data
